In [ ]:
import re
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
!unzip '/content/drive/MyDrive/AIOps/new_version/AIOps/sonar_results_AIOps.zip' -d /content/
!unzip '/content/drive/MyDrive/AIOps/new_version/ML/sonar_results_ML.zip' -d /content/
!unzip '/content/drive/MyDrive/AIOps/new_version/general/sonar_results_general.zip' -d /content/

In [ ]:
path_aiops = '/content/sonar_results_AIOps/'
path_ml = '/content/sonar_results_ML/'
path_general = '/content/sonar_results_general/'

# Extract code quality metrics

In [ ]:
empty_projects_aiops = 0
for project in os.listdir(path_aiops):
    all_aiops = pd.read_excel(path_aiops+project, sheet_name='All')
    security_hotspots_aiops = pd.read_excel(path_aiops+project, sheet_name='All')
    metrics_aiops = pd.read_excel(path_aiops+project, sheet_name='All')
    if(len(all_aiops) == 0 and len(security_hotspots_aiops) == 0 and len(metrics_aiops) == 0):
        empty_projects_aiops += 1
        print(project)
    # print(project, " ", len(all))
print(empty_projects_aiops)

2023-03-15-CloudWise-OpenSource-GAIA-DataSet-issues-report.xlsx
2023-03-15-NetManAIOps-AIOps-Challenge-2020-Data-issues-report.xlsx
2023-03-16-logpai-loghub-issues-report.xlsx
2023-03-16-alexfrancow-A-Detector-issues-report.xlsx
2023-03-15-centreon-centreon-issues-report.xlsx
2023-03-15-NetManAIOps-MultiDimension-Localization-issues-report.xlsx
2023-03-16-FedML-AI-FedIoT-issues-report.xlsx
2023-03-15-NetManAIOps-KPI-Anomaly-Detection-issues-report.xlsx
2023-03-15-sea-boat-Sea-AIOps-issues-report.xlsx
9


In [ ]:
empty_projects_ml = 0
for project in os.listdir(path_ml):
    all_ml = pd.read_excel(path_ml+project, sheet_name='All')
    security_hotspots_ml = pd.read_excel(path_ml+project, sheet_name='Security Hotspots')
    metrics_ml = pd.read_excel(path_ml+project, sheet_name='Metrics')
    if(len(all_ml) == 0 and len(security_hotspots_ml) == 0 and len(metrics_ml) == 0):
        empty_projects_ml += 1
        # print(project)
    # print(project, " ", len(all))
print(empty_projects_ml)

28


In [ ]:
empty_projects_general = 0
for project in os.listdir(path_general):
    all_general = pd.read_excel(path_general+project, sheet_name='All')
    security_hotspots_general = pd.read_excel(path_general+project, sheet_name='Security Hotspots')
    metrics_general = pd.read_excel(path_general+project, sheet_name='Metrics')
    if(len(all_general) == 0 and len(security_hotspots_general) == 0 and len(metrics_general) == 0):
        empty_projects_general += 1
        # print(project)
    # print(project, " ", len(all))
print(empty_projects_general)

76


In [ ]:
def bcv(nonokang_all, all_metrics):
    # Number of smells, bugs, and vulnerabilities
    try:
        all_metrics.append(len(nonokang_all[nonokang_all['type']=='CODE_SMELL']))
        all_metrics.append(len(nonokang_all[nonokang_all['type']=='BUG']))
        all_metrics.append(len(nonokang_all[nonokang_all['type']=='VULNERABILITY']))
    except:
        all_metrics.extend([0, 0, 0])
    
    return all_metrics

In [ ]:
def bcv_security(nonokang_security_hotspots, all_metrics):
    # Number of security hotspots
    try:
        all_metrics.append(len(nonokang_security_hotspots))
    except:
        all_metrics.append(0)
    
    return all_metrics

In [ ]:
def size(nonokang_metrics, all_metrics):
    try:
        # LOC, comment_lines, comment density
        i = 0
        files = 0
        ncloc = 0
        comment = 0
        duplicate = 0
        for index in nonokang_metrics['Path']:
            if '.' in nonokang_metrics['Path'][i]:
                ncloc_now = int(nonokang_metrics['ncloc'][i])
                ncloc += ncloc_now
                comment_percent_now = float(nonokang_metrics['comment_lines_density'][i])
                duplicate_percent_now = float(nonokang_metrics['duplicated_lines_density'][i])
                if comment_percent_now != 100:
                    comment_line_now = int((ncloc_now * comment_percent_now) / (100 - comment_percent_now))
                    comment += comment_line_now
                files += 1
            i += 1
        comment_density = (comment / (comment + ncloc)) * 100
        all_metrics.append(ncloc)
        all_metrics.append(comment)
        all_metrics.append(comment_density)
      
    except:
        all_metrics.append(0)

    return all_metrics

In [ ]:
def debt(nonokang_all, all_metrics):
    try:
        # debt time
        debt = 0
        i = 0
        for index in nonokang_all['debt']:
            if 'min' in nonokang_all['debt'][i] and 'h' in nonokang_all['debt'][i] and 'd' in nonokang_all['debt'][i]:
                debt_now = (int(nonokang_all['debt'][i].split('d')[0]) * 8 * 60) + (int(nonokang_all['debt'][i].split('d')[1].split('h')[0]) * 60) + int(nonokang_all['debt'][i].split('h')[1].split('min')[0])
            elif 'h' in nonokang_all['debt'][i] and 'd' in nonokang_all['debt'][i]:
                debt_now = (int(nonokang_all['debt'][i].split('d')[0]) * 8 * 60) + (int(nonokang_all['debt'][i].split('d')[1].split('h')[0]) * 60)
            elif 'min' in nonokang_all['debt'][i] and 'd' in nonokang_all['debt'][i]:
                debt_now = (int(nonokang_all['debt'][i].split('d')[0]) * 8 * 60) + int(nonokang_all['debt'][i].split('d')[1].split('min')[0])
            elif 'min' in nonokang_all['debt'][i] and 'h' in nonokang_all['debt'][i]:
                debt_now = (int(nonokang_all['debt'][i].split('h')[0]) * 60) + int(nonokang_all['debt'][i].split('h')[1].split('min')[0])
            elif 'd' in nonokang_all['debt'][i]:
                debt_now = (int(nonokang_all['debt'][i].split('d')[0]) * 8 * 60)
            elif 'h' in nonokang_all['debt'][i]:
                debt_now = (int(nonokang_all['debt'][i].split('h')[0]) * 60)
            elif 'min' in nonokang_all['debt'][i]:
                debt_now = int(nonokang_all['debt'][i].split('min')[0])
            else:
                debt_now = 0

            debt += debt_now
            i += 1

        all_metrics.append(debt)
    except:
        all_metrics.append(0)
    
    return all_metrics

In [ ]:
def severities(nonokang_all, all_metrics):
    try:
        code_smells = nonokang_all[nonokang_all['type'] == "CODE_SMELL"]
        bugs = nonokang_all[nonokang_all['type'] == "BUG"]
        vulnerabilities = nonokang_all[nonokang_all['type'] == "VULNERABILITY"]

        all_metrics.append(len(code_smells[code_smells['severity'] == 'MINOR']))
        all_metrics.append(len(code_smells[code_smells['severity'] == 'MAJOR']))
        all_metrics.append(len(code_smells[code_smells['severity'] == 'CRITICAL']))
        all_metrics.append(len(code_smells[code_smells['severity'] == 'BLOCKER']))
        all_metrics.append(len(code_smells[code_smells['severity'] == 'INFO']))

        all_metrics.append(len(bugs[bugs['severity'] == 'MINOR']))
        all_metrics.append(len(bugs[bugs['severity'] == 'MAJOR']))
        all_metrics.append(len(bugs[bugs['severity'] == 'CRITICAL']))
        all_metrics.append(len(bugs[bugs['severity'] == 'BLOCKER']))
        all_metrics.append(len(bugs[bugs['severity'] == 'INFO']))

        all_metrics.append(len(vulnerabilities[vulnerabilities['severity'] == 'MINOR']))
        all_metrics.append(len(vulnerabilities[vulnerabilities['severity'] == 'MAJOR']))
        all_metrics.append(len(vulnerabilities[vulnerabilities['severity'] == 'CRITICAL']))
        all_metrics.append(len(vulnerabilities[vulnerabilities['severity'] == 'BLOCKER']))
        all_metrics.append(len(vulnerabilities[vulnerabilities['severity'] == 'INFO']))
    except:
        for i in range(15):
            all_metrics.append(0)

    return all_metrics

In [ ]:
def severities_security(nonokang_security_hotspots, all_metrics):
    try:
        all_metrics.append(len(nonokang_security_hotspots[nonokang_security_hotspots['Severity'] == 'MINOR']))
        all_metrics.append(len(nonokang_security_hotspots[nonokang_security_hotspots['Severity'] == 'MAJOR']))
        all_metrics.append(len(nonokang_security_hotspots[nonokang_security_hotspots['Severity'] == 'CRITICAL']))
        all_metrics.append(len(nonokang_security_hotspots[nonokang_security_hotspots['Severity'] == 'BLOCKER']))
        all_metrics.append(len(nonokang_security_hotspots[nonokang_security_hotspots['Severity'] == 'INFO']))
    except:
        all_metrics.extend([0, 0, 0, 0, 0])
        
    return all_metrics

In [ ]:
def create_metrics(path):
    i = 0
    j = 0
    all_metrics_final = pd.DataFrame()
    for project in os.listdir(path):
        # nonokang_issues = pd.read_excel(path+project, sheet_name='Issues')
        nonokang_all = pd.read_excel(path+project, sheet_name='All')
        nonokang_security_hotspots = pd.read_excel(path+project, sheet_name='Security Hotspots')
        nonokang_metrics = pd.read_excel(path+project, sheet_name='Metrics')

        nonokang_all.fillna(0,inplace=True)   
        nonokang_security_hotspots.fillna(0,inplace=True)
        nonokang_metrics.fillna(0,inplace=True)

        if(len(nonokang_all) == 0 and len(nonokang_security_hotspots) == 0 and len(nonokang_metrics) == 0):
            i+=1
        else:
            try:
                all_metrics = []
                all_metrics = bcv(nonokang_all, all_metrics)
                all_metrics = bcv_security(nonokang_security_hotspots, all_metrics)
                all_metrics = size(nonokang_metrics, all_metrics)
                
                all_metrics = debt(nonokang_all, all_metrics)
                
                all_metrics = severities(nonokang_all, all_metrics)
                
                all_metrics = severities_security(nonokang_security_hotspots, all_metrics)
                
                all_metrics_now = pd.DataFrame(all_metrics)
                all_metrics_final = all_metrics_final.append(all_metrics_now.T)
            except:
                print("error2", project)
                j += 1
                pass
    print(i, j)
    all_metrics_final.columns = ['smells', 'bugs', 'vulnerabilities', 'security_hotspots', 'lines_of_code', 'comments', 'comments_density', 'debt',
                            'smells_minor', 'smells_major', 'smells_critical', 'smells_blocker', 'smells_info', 
                            'bugs_minor', 'bugs_major', 'bugs_critical', 'bugs_blocker', 'bugs_info', 
                            'vulnerabilities_minor', 'vulnerabilities_major', 'vulnerabilities_critical', 'vulnerabilities_blocker', 'vulnerabilities_info', 
                            'hotspots_minor', 'hotspots_major', 'hotspots_critical', 'hotspots_blocker', 'hotspots_info']
    return all_metrics_final

In [ ]:
all_metrics_aiops = create_metrics(path_aiops)
all_metrics_ml = create_metrics(path_ml)
all_metrics_general = create_metrics(path_general)

8 0
28 0
76 0


In [ ]:
all_metrics_aiops.head()

,smells,bugs,vulnerabilities,security_hotspots,lines_of_code,comments,comments_density,debt,smells_minor,smells_major,...,vulnerabilities_minor,vulnerabilities_major,vulnerabilities_critical,vulnerabilities_blocker,vulnerabilities_info,hotspots_minor,hotspots_major,hotspots_critical,hotspots_blocker,hotspots_info
0,7.0,0.0,0.0,2.0,2411.0,107.0,4.249404,38.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
0,40.0,0.0,0.0,0.0,2165.0,292.0,11.884412,405.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,86.0,0.0,0.0,1.0,1825.0,1319.0,41.952926,584.0,39.0,31.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0,143.0,7.0,0.0,2.0,2445.0,1577.0,39.209349,810.0,77.0,26.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
0,30.0,4.0,0.0,2.0,238.0,108.0,31.213873,176.0,20.0,10.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [ ]:
# Save results to csv
all_metrics_aiops.to_csv('aiops_rq3_raw.csv', index=False)
all_metrics_ml.to_csv('ml_rq3_raw.csv', index=False)
all_metrics_general.to_csv('general_rq3_raw.csv', index=False)

# Rules and Tags

In [ ]:
# rules for all projects - AIOps - final version
rules_all_aiops = pd.Series([])
for project in os.listdir(path_aiops):
    try:
        # print(project)
        all_aiops = pd.read_excel(path_aiops+project, sheet_name='All')
        rules_now_aiops = all_aiops['rule']
        # print(rules_now)
        rules_all_aiops = rules_all_aiops.append(rules_now_aiops)
    except:
        pass
print(rules_all_aiops.value_counts().sum())
rules_all_aiops.value_counts()[:30]

53239


Web:S1827                         8422
go:S100                           5461
python:S117                       2824
java:S117                         1709
Web:TableWithoutCaptionCheck      1394
go:S1186                          1385
Web:S5256                         1312
go:S1192                          1286
python:S125                       1284
go:S3776                          1229
javascript:S1117                  1165
java:S2293                         828
java:S1068                         811
java:S1450                         746
python:S1192                       737
java:S125                          564
python:PrintStatementUsage         534
javascript:S1121                   523
javascript:S3776                   514
python:S1542                       502
python:S1481                       498
css:S4666                          496
java:S100                          488
python:S3776                       482
java:S120                          474
Web:AvoidCommentedOutCode

In [ ]:
# rules for all projects - AIOps - final version
rules_all_aiops = pd.DataFrame()
for project in os.listdir(path_aiops):
    try:
        all_aiops = pd.read_excel(path_aiops+project, sheet_name='All')
        rules_now_aiops = all_aiops['rule']
        ff = rules_now_aiops.value_counts()/rules_now_aiops.value_counts().sum()
        df = pd.DataFrame({'rule':ff.index, 'amount':ff.values})
        rules_all_aiops = rules_all_aiops.append(df, ignore_index=True)
        # break
    except:
        # break
        pass

dff2_serie = rules_all_aiops['rule'].value_counts()
dff2 = pd.DataFrame({'rule':dff2_serie.index, 'count':dff2_serie.values})
dff2['count'] = (dff2['count'] / (len(os.listdir(path_aiops)) - empty_projects_aiops) * 100)
dff = rules_all_aiops.groupby(["rule"])['amount'].sum().reset_index()
dff['amount'] = (dff['amount'] / (len(os.listdir(path_aiops)) - empty_projects_aiops) * 100)
dff = dff.sort_values('amount', ascending=False)
df_final_rule_aiops = pd.merge(dff, dff2, on='rule')
df_final_rule_aiops

,rule,amount,count
0,python:S117,17.643147,59.090909
1,python:S125,14.646015,66.363636
2,python:S1192,7.808371,67.272727
3,python:S905,6.978401,25.454545
4,python:S1481,4.151497,53.636364
5,python:S3776,3.347383,48.181818
6,python:S1542,3.151234,44.545455
7,python:S2208,2.294806,25.454545
8,python:S5754,2.153570,35.454545
9,python:S100,2.110711,20.909091


In [ ]:
# tags for all projects - AIOps - final version
tags_all_aiops = pd.DataFrame()
for project in os.listdir(path_aiops):
    try:
        b = []
        i = 0
        all_tags = pd.read_excel(path_aiops+project, sheet_name='All')
        for index in all_tags['tags']:
            a = []
            if '[' in index and ']' in index:
                tag = all_tags['tags'][i][1:-1].replace(" ", "")
            else:
                tag = all_tags['tags'][i].replace(" ", "")
            if ',' in tag:
                a = tag.split(',')
            else:
                a.append(tag)
            b.extend(a)
            i += 1
        tags_now_aiops = pd.Series(b)
        ff = tags_now_aiops.value_counts()/tags_now_aiops.value_counts().sum()
        df = pd.DataFrame({'tag':ff.index, 'amount':ff.values})
        tags_all_aiops = tags_all_aiops.append(df, ignore_index=True)
    except:
        pass

dff2_serie = tags_all_aiops['tag'].value_counts()
dff2 = pd.DataFrame({'tag':dff2_serie.index, 'count':dff2_serie.values})
dff2['count'] = (dff2['count'] / (len(os.listdir(path_aiops)) - empty_projects_aiops) * 100)
dff = tags_all_aiops.groupby(["tag"])['amount'].sum().reset_index()
dff['amount'] = (dff['amount'] / (len(os.listdir(path_aiops)) - empty_projects_aiops) * 100)
dff = dff.sort_values('amount', ascending=False)

df_final_tag_aiops = pd.merge(dff, dff2, on='tag')
df_final_tag_aiops

,tag,amount,count
0,convention,25.726231,76.363636
1,unused,24.187824,93.636364
2,design,9.060665,79.090909
3,cwe,8.289216,66.363636
4,suspicious,5.592335,66.363636
5,brain-overload,4.218582,60.909091
6,pitfall,3.328834,42.727273
7,,3.142233,44.545455
8,error-handling,2.063599,49.090909
9,bad-practice,1.985513,51.818182


In [ ]:
# rules for all projects - ML - final version
rules_all_ml = pd.DataFrame()
for project in os.listdir(path_ml):
    try:
        all_ml = pd.read_excel(path_ml+project, sheet_name='All')
        rules_now_ml = all_ml['rule']
        ff = rules_now_ml.value_counts()/rules_now_ml.value_counts().sum()
        df = pd.DataFrame({'rule':ff.index, 'amount':ff.values})
        rules_all_ml = rules_all_ml.append(df, ignore_index=True)
        # break
    except:
        # break
        pass

dff2_serie = rules_all_ml['rule'].value_counts()
dff2 = pd.DataFrame({'rule':dff2_serie.index, 'count':dff2_serie.values})
dff2['count'] = (dff2['count'] / (len(os.listdir(path_ml)) - empty_projects_ml) * 100)
dff = rules_all_ml.groupby(["rule"])['amount'].sum().reset_index()
dff['amount'] = (dff['amount'] / (len(os.listdir(path_ml)) - empty_projects_ml) * 100)
dff = dff.sort_values('amount', ascending=False)

df_final_rule_ml = pd.merge(dff, dff2, on='rule')
df_final_rule_ml

,rule,amount,count
0,python:S117,16.792521,59.322034
1,python:S125,15.474323,63.841808
2,python:S905,7.655036,27.683616
3,python:S1192,7.303626,55.367232
4,python:S1481,5.879140,52.542373
5,python:S1542,3.955074,40.960452
6,python:S3776,3.232986,43.502825
7,python:PrintStatementUsage,2.758001,8.757062
8,python:S2208,2.441535,23.446328
9,python:S5754,2.092639,25.423729


In [ ]:
# tags for all projects - ML - final version
tags_all_ml = pd.DataFrame()
for project in os.listdir(path_ml):
    try:
        b = []
        i = 0
        all_tags = pd.read_excel(path_ml+project, sheet_name='All')
        for index in all_tags['tags']:
            a = []
            if '[' in index and ']' in index:
                tag = all_tags['tags'][i][1:-1].replace(" ", "")
            else:
                tag = all_tags['tags'][i].replace(" ", "")
            if ',' in tag:
                a = tag.split(',')
            else:
                a.append(tag)
            b.extend(a)
            i += 1
        tags_now_ml = pd.Series(b)
        ff = tags_now_ml.value_counts()/tags_now_ml.value_counts().sum()
        df = pd.DataFrame({'tag':ff.index, 'amount':ff.values})
        tags_all_ml = tags_all_ml.append(df, ignore_index=True)
    except:
        pass

dff2_serie = tags_all_ml['tag'].value_counts()
dff2 = pd.DataFrame({'tag':dff2_serie.index, 'count':dff2_serie.values})
dff2['count'] = (dff2['count'] / (len(os.listdir(path_ml)) - empty_projects_ml) * 100)
dff = tags_all_ml.groupby(["tag"])['amount'].sum().reset_index()
dff['amount'] = (dff['amount'] / (len(os.listdir(path_ml)) - empty_projects_ml) * 100)
dff = dff.sort_values('amount', ascending=False)

df_final_tag_ml = pd.merge(dff, dff2, on='tag')
df_final_tag_ml

,tag,amount,count
0,unused,26.697229,83.615819
1,convention,21.863060,67.796610
2,cwe,6.674528,55.932203
3,design,6.512361,57.627119
4,suspicious,4.395718,48.022599
5,,3.450195,35.310734
6,pitfall,3.440503,39.830508
7,brain-overload,3.308479,49.152542
8,obsolete,2.004238,13.559322
9,bad-practice,1.996451,37.853107


In [ ]:
# rules for all projects - General - final version
rules_all_general = pd.DataFrame()
for project in os.listdir(path_general):
    try:
        all_general = pd.read_excel(path_general+project, sheet_name='All')
        rules_now_general = all_general['rule']
        ff = rules_now_general.value_counts()/rules_now_general.value_counts().sum()
        df = pd.DataFrame({'rule':ff.index, 'amount':ff.values})
        rules_all_general = rules_all_general.append(df, ignore_index=True)
        # break
    except:
        # break
        pass

dff2_serie = rules_all_general['rule'].value_counts()
dff2 = pd.DataFrame({'rule':dff2_serie.index, 'count':dff2_serie.values})
dff2['count'] = (dff2['count'] / (len(os.listdir(path_general)) - empty_projects_general) * 100)
dff = rules_all_general.groupby(["rule"])['amount'].sum().reset_index()
dff['amount'] = (dff['amount'] / (len(os.listdir(path_general)) - empty_projects_general) * 100)
dff = dff.sort_values('amount', ascending=False)

df_final_rule_general = pd.merge(dff, dff2, on='rule')
df_final_rule_general

,rule,amount,count
0,python:S117,5.230793,19.672131
1,python:S125,3.944199,20.327869
2,javascript:S1117,3.446567,19.344262
3,Web:S5254,3.045004,20.655738
4,python:S1192,2.611356,19.016393
5,javascript:S125,2.296283,16.721311
6,python:S1481,1.879327,18.032787
7,xml:S125,1.816751,5.245902
8,Web:S1827,1.683862,7.540984
9,javascript:S2703,1.674426,12.459016


In [ ]:
# tags for all projects - General - final version
tags_all_general = pd.DataFrame()
for project in os.listdir(path_general):
    try:
        b = []
        i = 0
        all_tags = pd.read_excel(path_general+project, sheet_name='All')
        for index in all_tags['tags']:
            a = []
            if '[' in index and ']' in index:
                tag = all_tags['tags'][i][1:-1].replace(" ", "")
            else:
                tag = all_tags['tags'][i].replace(" ", "")
            if ',' in tag:
                a = tag.split(',')
            else:
                a.append(tag)
            b.extend(a)
            i += 1
        tags_now_general = pd.Series(b)
        ff = tags_now_general.value_counts()/tags_now_general.value_counts().sum()
        df = pd.DataFrame({'tag':ff.index, 'amount':ff.values})
        tags_all_general = tags_all_general.append(df, ignore_index=True)
    except:
        pass

dff2_serie = tags_all_general['tag'].value_counts()
dff2 = pd.DataFrame({'tag':dff2_serie.index, 'count':dff2_serie.values})
dff2['count'] = (dff2['count'] / (len(os.listdir(path_general)) - empty_projects_general) * 100)
dff = tags_all_general.groupby(["tag"])['amount'].sum().reset_index()
dff['amount'] = (dff['amount'] / (len(os.listdir(path_general)) - empty_projects_general) * 100)
dff = dff.sort_values('amount', ascending=False)

df_final_tag_general = pd.merge(dff, dff2, on='tag')
df_final_tag_general

,tag,amount,count
0,unused,15.145321,62.295082
1,convention,10.819123,40.000000
2,suspicious,7.500184,52.459016
3,pitfall,7.460032,47.868852
4,cwe,6.060659,50.491803
5,design,5.070817,38.360656
6,accessibility,4.492294,29.180328
7,,4.053134,39.344262
8,brain-overload,3.580251,43.606557
9,wcag2-a,3.457157,27.540984


# Comparing project sets

**Default version**

In [ ]:
aiops = pd.read_csv('/content/drive/MyDrive/AIOps/new_version/AIOps/aiops_rq3_raw.csv')
ml = pd.read_csv('/content/drive/MyDrive/AIOps/new_version/ML/ml_rq3_raw.csv')
general = pd.read_csv('/content/drive/MyDrive/AIOps/new_version/general/general_rq3_raw.csv')

In [ ]:
all_mean = {
       'bugs':[aiops['bugs'].mean(), ml['bugs'].mean(), general['bugs'].mean()],
       'smells':[aiops['smells'].mean(), ml['smells'].mean(), general['smells'].mean()], 
       'vulnerabilities':[aiops['vulnerabilities'].mean(), ml['vulnerabilities'].mean(), general['vulnerabilities'].mean()],
       'security_hotspots':[aiops['security_hotspots'].mean(), ml['security_hotspots'].mean(), general['security_hotspots'].mean()],
       'lines_of_code':[aiops['lines_of_code'].mean(), ml['lines_of_code'].mean(), general['lines_of_code'].mean()],
       'comments':[aiops['comments'].mean(), ml['comments'].mean(), general['comments'].mean()],
       'comments_density':[aiops['comments_density'].mean(), ml['comments_density'].mean(), general['comments_density'].mean()],
       'debt(hour)':[aiops['debt'].mean()/60, ml['debt'].mean()/60, general['debt'].mean()/60]}
 
df_all_mean = pd.DataFrame(all_mean)
df_all_mean.index = ['AIOps', 'ML', 'general']
df_all_mean

,bugs,smells,vulnerabilities,security_hotspots,lines_of_code,comments,comments_density,debt(hour)
AIOps,50.477477,429.018018,0.135135,22.360360,21250.612613,2167.531532,20.379345,80.488138
ML,34.463277,103.923729,0.002825,5.070621,6982.135593,1145.101695,22.002018,26.786582
general,45.111475,337.239344,0.226230,17.855738,10711.711475,1494.622951,11.584355,57.269399


In [ ]:
all_median = {
       'bugs':[aiops['bugs'].median(), ml['bugs'].median(), general['bugs'].median()],
       'smells':[aiops['smells'].median(), ml['smells'].median(), general['smells'].median()], 
       'vulnerabilities':[aiops['vulnerabilities'].median(), ml['vulnerabilities'].median(), general['vulnerabilities'].median()],
       'security_hotspots':[aiops['security_hotspots'].median(), ml['security_hotspots'].median(), general['security_hotspots'].median()],
       'lines_of_code':[aiops['lines_of_code'].median(), ml['lines_of_code'].median(), general['lines_of_code'].median()],
       'comments':[aiops['comments'].median(), ml['comments'].median(), general['comments'].median()],
       'comments_density':[aiops['comments_density'].median(), ml['comments_density'].median(), general['comments_density'].median()],
       'debt(hour)':[aiops['debt'].median()/60, ml['debt'].median()/60, general['debt'].median()/60]}
 
df_all_median = pd.DataFrame(all_median)
df_all_median.index = ['AIOps', 'ML', 'general']
df_all_median

,bugs,smells,vulnerabilities,security_hotspots,lines_of_code,comments,comments_density,debt(hour)
AIOps,3.0,51.0,0.0,1.0,1315.0,285.0,18.963878,5.383333
ML,1.0,26.5,0.0,0.0,787.5,211.5,20.856747,2.916667
general,1.0,14.0,0.0,1.0,825.0,77.0,8.408408,1.666667


**Normalized version**

In [ ]:
aiops_normalized = pd.DataFrame()
ml_normalized = pd.DataFrame()
general_normalized = pd.DataFrame()

In [ ]:
aiops_normalized['bugs'] = aiops['bugs'] / aiops['lines_of_code'] * 1000
aiops_normalized['smells'] = aiops['smells'] / aiops['lines_of_code'] * 1000
aiops_normalized['vulnerabilities'] = aiops['vulnerabilities'] / aiops['lines_of_code'] * 1000
aiops_normalized['security_hotspots'] = aiops['security_hotspots'] / aiops['lines_of_code'] * 1000
aiops_normalized['debt'] = aiops['debt'] / aiops['lines_of_code'] * 1000
aiops_normalized = aiops_normalized.fillna(0)

ml_normalized['bugs'] = ml['bugs'] / ml['lines_of_code'] * 1000
ml_normalized['smells'] = ml['smells'] / ml['lines_of_code'] * 1000
ml_normalized['vulnerabilities'] = ml['vulnerabilities'] / ml['lines_of_code'] * 1000
ml_normalized['security_hotspots'] = ml['security_hotspots'] / ml['lines_of_code'] * 1000
ml_normalized['debt'] = ml['debt'] / ml['lines_of_code'] * 1000
ml_normalized = ml_normalized.fillna(0)

general_normalized['bugs'] = general['bugs'] / general['lines_of_code'] * 1000
general_normalized['smells'] = general['smells'] / general['lines_of_code'] * 1000
general_normalized['vulnerabilities'] = general['vulnerabilities'] / general['lines_of_code'] * 1000
general_normalized['security_hotspots'] = general['security_hotspots'] / general['lines_of_code'] * 1000
general_normalized['debt'] = general['debt'] / general['lines_of_code'] * 1000
general_normalized = general_normalized.fillna(0)

In [ ]:
# Save results to csv
aiops_normalized.to_csv('aiops_rq3_normalized.csv', index=False)
ml_normalized.to_csv('ml_rq3_normalized.csv', index=False)
general_normalized.to_csv('general_rq3_normalized.csv', index=False)

In [ ]:
all_mean_normalized = {
       'bugs':[aiops_normalized['bugs'].mean(), ml_normalized['bugs'].mean(), general_normalized['bugs'].mean()],
       'smells':[aiops_normalized['smells'].mean(), ml_normalized['smells'].mean(), general_normalized['smells'].mean()], 
       'vulnerabilities':[aiops_normalized['vulnerabilities'].mean(), ml_normalized['vulnerabilities'].mean(), general_normalized['vulnerabilities'].mean()],
       'security_hotspots':[aiops_normalized['security_hotspots'].mean(), ml_normalized['security_hotspots'].mean(), general_normalized['security_hotspots'].mean()],
       'debt':[aiops_normalized['debt'].mean()/60, ml_normalized['debt'].mean()/60, general_normalized['debt'].mean()/60]}
 
df_all_mean_normalized = pd.DataFrame(all_mean_normalized)
df_all_mean_normalized.index = ['AIOps', 'ML', 'general']
df_all_mean_normalized

,bugs,smells,vulnerabilities,security_hotspots,debt
AIOps,5.210463,52.479156,0.077893,1.953799,5.621820
ML,5.764245,40.451031,0.002350,2.372947,4.690126
general,4.739808,38.405746,0.045091,4.080891,4.983666


In [ ]:
all_median_normalized = {
       'bugs':[aiops_normalized['bugs'].median(), ml_normalized['bugs'].median(), general_normalized['bugs'].median()],
       'smells':[aiops_normalized['smells'].median(), ml_normalized['smells'].median(), general_normalized['smells'].median()], 
       'vulnerabilities':[aiops_normalized['vulnerabilities'].median(), ml_normalized['vulnerabilities'].median(), general_normalized['vulnerabilities'].median()],
       'security_hotspots':[aiops_normalized['security_hotspots'].median(), ml_normalized['security_hotspots'].median(), general_normalized['security_hotspots'].median()],
       'debt':[aiops_normalized['debt'].median()/60, ml_normalized['debt'].median()/60, general_normalized['debt'].median()/60]}
 
df_all_median_normalized = pd.DataFrame(all_median_normalized)
df_all_median_normalized.index = ['AIOps', 'ML', 'general']
df_all_median_normalized

,bugs,smells,vulnerabilities,security_hotspots,debt
AIOps,1.448226,31.847134,0.0,0.542005,4.660348
ML,0.434377,25.711276,0.0,0.024774,3.262366
general,0.386473,18.463811,0.0,0.227627,2.380952


**Types and severity of issues**

In [ ]:
def count_issues(set):
    minor = set['smells_minor'].sum() + set['bugs_minor'].sum() + set['vulnerabilities_minor'].sum() + set['hotspots_minor'].sum()
    major = set['smells_major'].sum() + set['bugs_major'].sum() + set['vulnerabilities_major'].sum() + set['hotspots_major'].sum()
    critical = set['smells_critical'].sum() + set['bugs_critical'].sum() + set['vulnerabilities_critical'].sum() + set['hotspots_critical'].sum()
    blocker = set['smells_blocker'].sum() + set['bugs_blocker'].sum() + set['vulnerabilities_blocker'].sum() + set['hotspots_blocker'].sum()
    info = set['smells_info'].sum() + set['bugs_info'].sum() + set['vulnerabilities_info'].sum() + set['hotspots_info'].sum()
    all_issues = minor + major + critical + blocker + info
    return minor, major, critical, blocker, info, all_issues

In [ ]:
# Severity of issues
minor_aiops, major_aiops, critical_aiops, blocker_aiops, info_aiops, all_issues_aiops = count_issues(aiops)
minor_ml, major_ml, critical_ml, blocker_ml, info_ml, all_issues_ml = count_issues(ml)
minor_general, major_general, critical_general, blocker_general, info_general, all_issues_general = count_issues(general)

print('AIOps:', f'{minor_aiops / all_issues_aiops * 100:.2f}', f'{major_aiops / all_issues_aiops * 100:.2f}', 
      f'{critical_aiops / all_issues_aiops * 100:.2f}', f'{blocker_aiops / all_issues_aiops * 100:.2f}', f'{info_aiops / all_issues_aiops * 100:.2f}')
print('ML:', f'{minor_ml / all_issues_ml * 100:.2f}', f'{major_ml / all_issues_ml * 100:.2f}', 
      f'{critical_ml / all_issues_ml * 100:.2f}', f'{blocker_ml / all_issues_ml * 100:.2f}', f'{info_ml / all_issues_ml * 100:.2f}')
print('General:', f'{minor_general / all_issues_general * 100:.2f}', f'{major_general / all_issues_general * 100:.2f}', 
      f'{critical_general / all_issues_general * 100:.2f}', f'{blocker_general / all_issues_general * 100:.2f}', f'{info_general / all_issues_general * 100:.2f}')

AIOps: 38.32 44.62 15.41 0.81 0.84
ML: 33.66 49.03 13.52 3.53 0.26
General: 33.16 52.15 12.87 0.78 1.05


In [ ]:
# Types of issues
print('AIOps:', round(aiops['bugs'].sum() / all_issues_aiops * 100, 2), round(aiops['smells'].sum() / all_issues_aiops * 100, 2), 
      round(aiops['vulnerabilities'].sum() / all_issues_aiops * 100, 2), round(aiops['security_hotspots'].sum() / all_issues_aiops * 100, 2))
print('ML:', round(ml['bugs'].sum() / all_issues_ml * 100, 2), round(ml['smells'].sum() / all_issues_ml * 100, 2), 
      round(ml['vulnerabilities'].sum() / all_issues_ml * 100, 2), round(ml['security_hotspots'].sum() / all_issues_ml * 100, 2))
print('General:', round(general['bugs'].sum() / all_issues_general * 100, 2), round(general['smells'].sum() / all_issues_general * 100, 2), 
      round(general['vulnerabilities'].sum() / all_issues_general * 100, 2), round(general['security_hotspots'].sum() / all_issues_general * 100, 2))

AIOps: 10.06 85.46 0.03 4.45
ML: 24.02 72.44 0.0 3.53
General: 11.27 84.22 0.06 4.46


# Statistical Tests

In [ ]:
aiops = pd.read_csv('/content/drive/MyDrive/AIOps/new_version/AIOps/aiops_rq3_results.csv')
ml = pd.read_csv('/content/drive/MyDrive/AIOps/new_version/ML/ml_rq3_results.csv')
general = pd.read_csv('/content/drive/MyDrive/AIOps/new_version/general/general_rq3_results.csv')

In [ ]:
# Normality test: Shapiro-Wilk Test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
def shapiro_wilk(d):
    data = d
    # seed the random number generator
    seed(1)
    # normality test
    stat, p = shapiro(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')

shapiro_wilk(aiops['smells'])
shapiro_wilk(aiops['bugs'])
shapiro_wilk(aiops['vulnerabilities'])
shapiro_wilk(aiops['security_hotspots'])
shapiro_wilk(aiops['lines_of_code'])
shapiro_wilk(aiops['comments'])
shapiro_wilk(aiops['comments_density'])
shapiro_wilk(aiops['debt'])
shapiro_wilk(ml['smells'])
shapiro_wilk(ml['bugs'])
shapiro_wilk(ml['vulnerabilities'])
shapiro_wilk(ml['security_hotspots'])
shapiro_wilk(ml['lines_of_code'])
shapiro_wilk(ml['comments'])
shapiro_wilk(ml['comments_density'])
shapiro_wilk(ml['debt'])
shapiro_wilk(general['smells'])
shapiro_wilk(general['bugs'])
shapiro_wilk(general['vulnerabilities'])
shapiro_wilk(general['security_hotspots'])
shapiro_wilk(general['lines_of_code'])
shapiro_wilk(general['comments'])
shapiro_wilk(general['comments_density'])
shapiro_wilk(general['debt'])

Statistics=0.313, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.138, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.164, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.266, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.281, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.295, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.952, p=0.001
Sample does not look Gaussian (reject H0)
Statistics=0.274, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.461, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.106, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.028, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.365, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.220, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.173, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.963, p=0.000
Sample does not look G

The distributions are NOT normall. Hence, we use non-parametric tests.

In [ ]:
# Statistical test: Mann-Whitney U Test (Wilcoxon rank-sum test)
from scipy.stats import mannwhitneyu
def whitney(d1, d2, par):
    stat, p = mannwhitneyu(d1, d2, alternative='two-sided')
    print('stat=%.2f, p=%.2f' % (stat, p))
    if p > 0.05:
        print(par, ': Probably same distribution')
    else:
        print(par, ': Probably DIFFERENT distributions')

whitney(aiops['bugs'], ml['bugs'], 'bugs')
whitney(aiops['bugs'], general['bugs'], 'bugs')
whitney(aiops['smells'], ml['smells'], 'smells')
whitney(aiops['smells'], general['smells'], 'smells')
whitney(aiops['vulnerabilities'], ml['vulnerabilities'], 'vulnerabilities')
whitney(aiops['vulnerabilities'], general['vulnerabilities'], 'vulnerabilities')
whitney(aiops['security_hotspots'], ml['security_hotspots'], 'security_hotspots')
whitney(aiops['security_hotspots'], general['security_hotspots'], 'security_hotspots')
whitney(aiops['debt'], ml['debt'], 'debt')
whitney(aiops['debt'], general['debt'], 'debt')
whitney(aiops['lines_of_code'], ml['lines_of_code'], 'lines_of_code')
whitney(aiops['lines_of_code'], general['lines_of_code'], 'lines_of_code')
whitney(aiops['comments'], ml['comments'], 'comments')
whitney(aiops['comments'], general['comments'], 'comments')
whitney(aiops['comments_density'], ml['comments_density'], 'comments_density')
whitney(aiops['comments_density'], general['comments_density'], 'comments_density')

stat=22557.50, p=0.01
bugs : Probably DIFFERENT distributions
stat=19240.00, p=0.03
bugs : Probably DIFFERENT distributions
stat=24194.50, p=0.00
smells : Probably DIFFERENT distributions
stat=22054.50, p=0.00
smells : Probably DIFFERENT distributions
stat=20301.50, p=0.00
vulnerabilities : Probably DIFFERENT distributions
stat=16926.50, p=1.00
vulnerabilities : Probably same distribution
stat=22121.00, p=0.03
security_hotspots : Probably DIFFERENT distributions
stat=18292.00, p=0.18
security_hotspots : Probably same distribution
stat=24477.50, p=0.00
debt : Probably DIFFERENT distributions
stat=21904.00, p=0.00
debt : Probably DIFFERENT distributions
stat=23019.50, p=0.01
lines_of_code : Probably DIFFERENT distributions
stat=19385.00, p=0.02
lines_of_code : Probably DIFFERENT distributions
stat=22823.50, p=0.01
comments : Probably DIFFERENT distributions
stat=23067.50, p=0.00
comments : Probably DIFFERENT distributions
stat=18640.50, p=0.42
comments_density : Probably same distributio

In [ ]:
!pip install cliffs-delta

In [ ]:
# Effect size test: Cliffs delta Test
from cliffs_delta import cliffs_delta
def cliff(d1, d2, par):
    d, res = cliffs_delta(d1, d2)
    print(par, ':', d,res)

cliff(aiops['bugs'], ml['bugs'], 'bugs')
cliff(aiops['bugs'], general['bugs'], 'bugs')
cliff(aiops['smells'], ml['smells'], 'smells')
cliff(aiops['smells'], general['smells'], 'smells')
cliff(aiops['vulnerabilities'], ml['vulnerabilities'], 'vulnerabilities')
cliff(aiops['vulnerabilities'], general['vulnerabilities'], 'vulnerabilities')
cliff(aiops['security_hotspots'], ml['security_hotspots'], 'security_hotspots')
cliff(aiops['security_hotspots'], general['security_hotspots'], 'security_hotspots')
cliff(aiops['debt'], ml['debt'], 'debt')
cliff(aiops['debt'], general['debt'], 'debt')
cliff(aiops['lines_of_code'], ml['lines_of_code'], 'lines_of_code')
cliff(aiops['lines_of_code'], general['lines_of_code'], 'lines_of_code')
cliff(aiops['comments'], ml['comments'], 'comments')
cliff(aiops['comments'], general['comments'], 'comments')
cliff(aiops['comments_density'], ml['comments_density'], 'comments_density')
cliff(aiops['comments_density'], general['comments_density'], 'comments_density')

bugs : 0.14813966508881762 small
bugs : 0.1366120218579235 negligible
smells : 0.23146027383315518 small
smells : 0.3028799291094373 small
vulnerabilities : 0.03331297399094009 negligible
vulnerabilities : -5.9075468911534485e-05 negligible
security_hotspots : 0.12592253270219372 negligible
security_hotspots : 0.0806084773297888 negligible
debt : 0.245864508576373 small
debt : 0.2939890710382514 small
lines_of_code : 0.17165470555301063 small
lines_of_code : 0.145177964850096 negligible
comments : 0.1616786277803227 small
comments : 0.36272337911682173 medium
comments_density : -0.05122919529699191 negligible
comments_density : 0.43683355486634173 medium
